In [13]:
import pandas as pd
import numpy as np
import os
import glob
import tempfile
from collections import defaultdict

In [14]:
def leer_secciones_unicas():
    """Lee la lista de secciones únicas desde secciones_unicas.csv"""
    df = pd.read_csv('secciones_unicas.csv')
    # Filtrar solo valores numéricos en la columna 'seccion'
    secciones = [int(s) for s in df['seccion'] if str(s).isdigit()]
    return secciones

In [15]:
def encontrar_archivo_seccion(seccion, directorio='DatosPorAGEB'):
    """Busca en qué archivo aparece una sección específica"""
    archivos = glob.glob(os.path.join(directorio, '*.xlsx'))
    
    for archivo in archivos:
        try:
            # Primero verificar qué hojas tiene el archivo
            xl_file = pd.ExcelFile(archivo)
            hojas_disponibles = xl_file.sheet_names
            print(f"  Hojas disponibles en {os.path.basename(archivo)}: {hojas_disponibles}")
            
            # Buscar una hoja que contenga 'seccion' en el nombre (case insensitive)
            hoja_secciones = None
            for hoja in hojas_disponibles:
                if 'seccion' in hoja.lower():
                    hoja_secciones = hoja
                    break
            
            if hoja_secciones is None:
                print(f"  No se encontró hoja de secciones en {archivo}")
                continue
            
            # Leer la hoja de secciones
            df = pd.read_excel(archivo, sheet_name=hoja_secciones)
            print(f"  Columnas en {hoja_secciones}: {list(df.columns)}")
            
            # Buscar la sección en el DataFrame
            columna_seccion = None
            for col in df.columns:
                if 'seccion' in col.lower():
                    columna_seccion = col
                    break
            
            if columna_seccion is None:
                print(f"  No se encontró columna de sección en {archivo}")
                continue
            
            if seccion in df[columna_seccion].values:
                return archivo
                    
        except Exception as e:
            print(f"Error leyendo {archivo}: {e}")
            continue
    
    return None

In [16]:
def extraer_ageb_del_nombre_archivo(nombre_archivo):
    """Extrae el AGEB del nombre del archivo"""
    # El formato es cpv2020_2111400010732.xlsx
    nombre_base = os.path.basename(nombre_archivo)
    ageb = nombre_base.replace('.xlsx', '')
    return ageb

In [17]:
# =============================================================================
# LIMPIEZA INICIAL GENERALIZADA
# =============================================================================

def limpiar_dataset_inicial(dataframe, debug=False):
    """
    Limpieza inicial generalizada: elimina filas sin datos electorales
    y prepara el dataset básico
    
    Parámetros:
    - archivo_csv: Ruta al archivo CSV
    - debug: Si mostrar información de debug
    
    Retorna:
    - DataFrame limpio con datos electorales válidos
    """
    if debug:
        print(f"🔄 LIMPIANDO: {archivo_csv}")
    
    # Cargar datos
    df = dataframe
    filas_originales = len(df)
    
    # Identificar columnas electorales de partidos (excluir metadatos)
    columnas_electorales_partidos = [
        col for col in df.columns 
        if col.startswith('ELECTORAL_') 
        and not any(x in col.upper() for x in [
            'ANIO', 'CASILLAS', 'CARGO', 'GANADOR',
            'NUM_VOTOS_VALIDOS', 'NUM_VOTOS_NULOS', 'NUM_VOTOS_CAN_NREG',
            'TOTAL_VOTOS', 'LISTA_NOMINAL'
        ])
    ]

    #Droppeando columna de cargo
    df = df.drop(columns=['ELECTORAL_CARGO'])
    
    if debug:
        print(f"Columnas de partidos encontradas: {len(columnas_electorales_partidos)}")
    
    # Convertir columnas electorales a numérico
    for col in columnas_electorales_partidos:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Filtrar filas que tienen al menos algunos votos válidos
    if columnas_electorales_partidos:
        # Calcular suma de votos por fila (sin NaN)
        df_partidos = df[columnas_electorales_partidos].fillna(0)
        suma_votos_por_fila = df_partidos.sum(axis=1)
        
        # Mantener filas con votos > 0
        mask_con_votos = suma_votos_por_fila > 0
        df_limpio = df[mask_con_votos].copy()
        
        # Llenar NaN con 0 en columnas electorales
        for col in columnas_electorales_partidos:
            df_limpio[col] = df_limpio[col].fillna(0)
    else:
        df_limpio = df.copy()
    
    filas_finales = len(df_limpio)
    
    if debug:
        print(f"Limpieza completada: {filas_originales} → {filas_finales} filas")
        print(f"Eficiencia: {(filas_finales/filas_originales)*100:.1f}%")
    
    return df_limpio

In [18]:
# =============================================================================
# CÁLCULO DE MÉTRICAS ELECTORALES GENERALIZADO
# =============================================================================

def calcular_metricas_electorales(df, debug=False):
    """
    Calcula métricas electorales: participación, competitividad, fragmentación
    """
    if debug:
        print("Calculando métricas electorales...")
    
    df_con_metricas = df.copy()
    
    # Identificar columnas electorales
    columnas_partidos = [
        col for col in df.columns 
        if col.startswith('ELECTORAL_') 
        and not any(x in col.upper() for x in [
            'ANIO', 'CASILLAS', 'CARGO', 'GANADOR',
            'NUM_VOTOS_VALIDOS', 'NUM_VOTOS_NULOS', 'NUM_VOTOS_CAN_NREG',
            'TOTAL_VOTOS', 'LISTA_NOMINAL'
        ])
    ]
    
    # MÉTRICA 1: PARTICIPACIÓN
    if 'ELECTORAL_TOTAL_VOTOS' in df.columns and 'ELECTORAL_LISTA_NOMINAL' in df.columns:
        total_votos = pd.to_numeric(df_con_metricas['ELECTORAL_TOTAL_VOTOS'], errors='coerce')
        lista_nominal = pd.to_numeric(df_con_metricas['ELECTORAL_LISTA_NOMINAL'], errors='coerce')
        
        # Evitar división por cero
        mask_valido = (lista_nominal > 0) & (total_votos >= 0)
        df_con_metricas['PARTICIPACION_PCT'] = 0.0
        df_con_metricas.loc[mask_valido, 'PARTICIPACION_PCT'] = (
            total_votos[mask_valido] / lista_nominal[mask_valido]
        ) * 100
    
    # MÉTRICAS 2 y 3: COMPETITIVIDAD Y FRAGMENTACIÓN
    if len(columnas_partidos) >= 2:
        # Preparar datos numéricos
        datos_partidos = df_con_metricas[columnas_partidos].copy()
        for col in columnas_partidos:
            datos_partidos[col] = pd.to_numeric(datos_partidos[col], errors='coerce').fillna(0)
        
        # Ordenar votos por fila para obtener 1° y 2° lugar
        votos_ordenados = np.sort(datos_partidos.values, axis=1)
        primer_lugar = votos_ordenados[:, -1]
        segundo_lugar = votos_ordenados[:, -2] if votos_ordenados.shape[1] > 1 else np.zeros(len(votos_ordenados))
        total_votos_partidos = datos_partidos.sum(axis=1)
        
        # COMPETITIVIDAD: diferencia entre 1° y 2° lugar
        with np.errstate(divide='ignore', invalid='ignore'):
            competitividad = np.where(
                total_votos_partidos > 0,
                ((primer_lugar - segundo_lugar) / total_votos_partidos) * 100,
                0
            )
        df_con_metricas['COMPETITIVIDAD'] = competitividad
        
        # FRAGMENTACIÓN: índice de fraccionamiento
        with np.errstate(divide='ignore', invalid='ignore'):
            proporcion_votos = datos_partidos.values / total_votos_partidos.values.reshape(-1, 1)
            proporcion_votos = np.where(np.isfinite(proporcion_votos), proporcion_votos, 0)
            fragmentacion = 1 - np.sum(proporcion_votos**2, axis=1)
        df_con_metricas['FRAGMENTACION'] = fragmentacion
    
    if debug:
        print(f"Métricas calculadas para {len(columnas_partidos)} partidos")
        if 'PARTICIPACION_PCT' in df_con_metricas.columns:
            participacion = df_con_metricas['PARTICIPACION_PCT']
            print(f"Participación: μ={participacion.mean():.1f}%, σ={participacion.std():.1f}%")
    
    return df_con_metricas

In [19]:
def crear_dataset_por_tipo_eleccion(tipo_eleccion, df_electoral, df_ageb, mapeo_seccion_ageb, secciones_unicas):
    """Crea un dataset específico para un tipo de elección"""
    
    print(f"\nCreando dataset para {tipo_eleccion}...")
    
    # Filtrar datos electorales por tipo de elección
    df_electoral_filtrado = df_electoral[df_electoral['CARGO'] == tipo_eleccion]
    print(f"Total de registros para {tipo_eleccion}: {len(df_electoral_filtrado)}")
    
    datos_finales = []
    
    for seccion in secciones_unicas:
        if seccion in mapeo_seccion_ageb:
            ageb = mapeo_seccion_ageb[seccion]
            
            # Buscar la información demográfica del AGEB
            info_ageb = df_ageb[df_ageb['AGEB'] == ageb]
            
            if not info_ageb.empty:
                # Obtener la primera fila (debería ser única)
                fila_ageb = info_ageb.iloc[0]
                
                # Crear diccionario con sección y toda la información demográfica
                dato_seccion = {'SECCION': seccion, 'AGEB': ageb}
                
                # Agregar todas las columnas demográficas
                for columna in df_ageb.columns:
                    if columna != 'AGEB':
                        dato_seccion[columna] = fila_ageb[columna]
                
                # Buscar información electoral para esta sección y tipo de elección
                info_electoral = df_electoral_filtrado[df_electoral_filtrado['SECCION'] == seccion]
                
                if not info_electoral.empty:
                    # Tomar el registro más reciente (último año disponible)
                    fila_electoral = info_electoral.iloc[-1]
                    
                    # Agregar información electoral
                    columnas_electorales = ['ANIO', 'CASILLAS', 'CARGO', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 
                                          'MC', 'MORENA', 'PAN_PRD', 'PT_MORENA', 'PCPP', 'PSI', 'NA', 'ES',
                                          'NUM_VOTOS_VALIDOS', 'NUM_VOTOS_CAN_NREG', 'NUM_VOTOS_NULOS', 
                                          'TOTAL_VOTOS', 'LISTA_NOMINAL', 'GANADOR']
                    
                    for columna in columnas_electorales:
                        if columna in fila_electoral:
                            dato_seccion[f'ELECTORAL_{columna}'] = fila_electoral[columna]
                    
                    print(f"  Seccion {seccion}: AGEB {ageb} + datos {tipo_eleccion}")
                else:
                    print(f"  Seccion {seccion}: AGEB {ageb} (sin datos {tipo_eleccion})")
                
                datos_finales.append(dato_seccion)
            else:
                print(f"  No se encontro informacion demografica para AGEB {ageb}")
        else:
            print(f"  Seccion {seccion} sin AGEB asignado")
    
    # Crear DataFrame final
    df_final = pd.DataFrame(datos_finales)
    
    # Guardar el dataset
    nombre_archivo = f'Dataset_Secciones_{tipo_eleccion.replace("_", "_").upper()}.csv'
    nombre_archivo_test = f'Dataset_Secciones_{tipo_eleccion.replace("_", "_").upper()}_TEST.csv'

    df_final = limpiar_dataset_inicial(df_final)
    df_final = calcular_metricas_electorales(df_final)

    #df_final = 90% de las secciones
    df_final_train = df_final.head(int(len(df_final) * 0.90))
    df_final_test = df_final.tail(int(len(df_final) * 0.10))

    df_final_train.to_csv(nombre_archivo, index=False)
    df_final_test.to_csv(nombre_archivo_test, index=False)

    print(f"Dataset {tipo_eleccion} creado: {nombre_archivo}")
    print(f"Total de secciones con datos: {len(df_final)}")
    print(f"Columnas: {len(df_final.columns)}")
    
    return df_final

In [20]:
def crear_dataset_pipeline():
    """Lleva a cabo todas las funciones para crear el dataset final"""
    
    # Leer secciones únicas
    print("Leyendo secciones unicas...")
    secciones_unicas = leer_secciones_unicas()
    print(f"Total de secciones unicas: {len(secciones_unicas)}")
    
    # Leer dataset de AGEB con información demográfica
    print("Leyendo dataset de AGEB...")
    df_ageb = pd.read_csv('AGEB_Consolidado_Completo.csv')
    print(f"Total de AGEB en dataset: {len(df_ageb)}")
    
    # Leer dataset electoral
    print("Leyendo dataset electoral...")
    df_electoral = pd.read_csv('pue_11_2015_no_rp.csv')
    print(f"Total de registros electorales: {len(df_electoral)}")
    
    # Mostrar tipos de elección disponibles
    tipos_eleccion = df_electoral['CARGO'].unique()
    print(f"Tipos de elección disponibles: {tipos_eleccion}")
    
    # Diccionario para mapear sección -> AGEB
    mapeo_seccion_ageb = {}
    
    # Procesar cada sección para obtener el mapeo AGEB
    print("Procesando secciones para mapeo AGEB...")
    for i, seccion in enumerate(secciones_unicas):
        print(f"Procesando seccion {seccion} ({i+1}/{len(secciones_unicas)})")
        
        # Buscar en qué archivo aparece la sección
        archivo = encontrar_archivo_seccion(seccion)
        
        if archivo:
            # Extraer AGEB del nombre del archivo
            ageb = extraer_ageb_del_nombre_archivo(archivo)
            mapeo_seccion_ageb[seccion] = ageb
            print(f"  Seccion {seccion} -> AGEB {ageb}")
        else:
            print(f"  No se encontro archivo para seccion {seccion}")
    # Crear datasets separados por tipo de elección
    datasets_creados = {}
    
    for tipo_eleccion in tipos_eleccion:
        df_resultado = crear_dataset_por_tipo_eleccion(
            tipo_eleccion, df_electoral, df_ageb, mapeo_seccion_ageb, secciones_unicas
        )
        datasets_creados[tipo_eleccion] = df_resultado
    
    print(f"\nResumen final:")
    print(f"Se crearon {len(datasets_creados)} datasets:")
    for tipo, df in datasets_creados.items():
        print(f"  - {tipo}: {len(df)} secciones, {len(df.columns)} columnas")
    
    return datasets_creados

In [21]:
crear_dataset_pipeline()

Leyendo secciones unicas...
Total de secciones unicas: 166
Leyendo dataset de AGEB...
Total de AGEB en dataset: 122
Leyendo dataset electoral...
Total de registros electorales: 795
Tipos de elección disponibles: ['diputacion_federal_mr' 'presidencia' 'senaduria_mr']
Procesando secciones para mapeo AGEB...
Procesando seccion 1163 (1/166)
  Hojas disponibles en cpv2020_211140001079A.xlsx: ['Población', 'Secciones', 'Vivienda', 'Fecundidad', 'Mortalidad', 'Migración', 'Etnicidad', 'Discapacidad', 'Educación', 'Características económicas', 'Servicios de salud', 'Situación conyugal', 'Hogares censales', 'Religión']
  Columnas en Secciones: ['Secciones']
  Hojas disponibles en cpv2020_2111400010836.xlsx: ['Población', 'Secciones', 'Vivienda', 'Fecundidad', 'Mortalidad', 'Migración', 'Etnicidad', 'Discapacidad', 'Educación', 'Características económicas', 'Servicios de salud', 'Situación conyugal', 'Hogares censales', 'Religión']
  Columnas en Secciones: ['Secciones']
  Hojas disponibles en cp

{'diputacion_federal_mr':      SECCION                   AGEB  RELACION_HOMBRES_MUJERES  \
 0       1163  cpv2020_2111400015305                      93.4   
 1       1165  cpv2020_211140001079A                      87.3   
 2       1166  cpv2020_211140001079A                      87.3   
 3       1167  cpv2020_211140001079A                      87.3   
 4       1168  cpv2020_2111400010836                      90.6   
 ..       ...                    ...                       ...   
 158     2668  cpv2020_2111400015930                      90.8   
 159     2669  cpv2020_2111400015682                     105.2   
 160     2670  cpv2020_2111400015682                     105.2   
 161     2671  cpv2020_2111400016619                      94.9   
 162     2699  cpv2020_2111400015536                      85.5   
 
      RAZON_DE_DEPENDENCIA_TOTAL  RAZON_DE_DEPENDENCIA_INFANTIL  \
 0                          47.7                           23.9   
 1                          50.2               